# Analytics Avengers: Water Levels

In [3]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

# File to Load
San_Francisco_path = Path(.."Data/Analytics_Avengers-SF_CA_Sea_Levels.csv")
San_Diego_path = Path(.."Data/Analytics_Avengers-San_Diego_CA_Sea_Levels.csv")
Key_West_path = Path(.."Data/Analytics_Avengers-Key_West_FL_Sea_Level.csv")
Honolulu_path = Path(.."Data/Analytics_Avengers-Honolulu_Hi_Sea_Levels.csv")
Charleston_path = Path(.."Data/Analytics_Avengers-Charleston_SC_Sea_Level.csv")
Boston_path = Path(.."Data/Analytics_Avengers-Boston_MA_Sea_Levels.csv")
Seattle_path = Path(.."Data/Analytics_Avengers-Seattle_WA_Sea_Level.csv")

# Read Data Files and store them into Pandas DataFrames
San_Francisco = pd.read_csv(San_Francisco_path)
San_Diego = pd.read_csv(San_Diego_path)
Key_West = pd.read_csv(Key_West_path)
Honolulu = pd.read_csv(Honolulu_path)
Charleston = pd.read_csv(Charleston_path)
Boston = pd.read_csv(Boston_path)
Seattle = pd.read_csv(Seattle_path)

# Display the first few rows of the San Francisco DataFrame
San_Francisco.head()


SyntaxError: invalid syntax (907029512.py, line 8)

In [ ]:
San_Diego.head()

In [ ]:
Key_West.head()

In [ ]:
Honolulu.head()

In [ ]:
Charleston.head()

In [ ]:
Boston.head()

In [ ]:
Seattle.head()

In [ ]:
# List of file paths
file_paths = [
    "Data/Analytics_Avengers-SF_CA_Sea_Levels.csv",
    "Data/Analytics_Avengers-San_Diego_CA_Sea_Levels.csv",
    "Data/Analytics_Avengers-Key_West_FL_Sea_Level.csv",
    "Data/Analytics_Avengers-Honolulu_Hi_Sea_Levels.csv",
    "Data/Analytics_Avengers-Charleston_SC_Sea_Level.csv",
    "Data/Analytics_Avengers-Boston_MA_Sea_Levels.csv",
    "Data/Analytics_Avengers-Seattle_WA_Sea_Level.csv"
]

# Create an empty list to store DataFrames
dataframes = []

# Loop through the file paths and read each CSV file into a DataFrame
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenate all DataFrames vertically into one
merged_data = pd.concat(dataframes, ignore_index=True)
merged_data

In [ ]:
# Drop unnecessary columns
columns_to_drop = ['Time (GMT)', 'Highest', 'MHHW (ft)', 'MTL (ft)', 'MLLW (ft)', 'Lowest (ft)', 'Inf']
merged_data_new = merged_data.drop(columns=columns_to_drop)

# Display the first few rows of the DataFrame to confirm the columns have been dropped
merged_data_new

In [ ]:
# Check for missing values in the DataFrame
missing_values = merged_data_new.isnull().sum()

# Print missing values before filling
print("Missing Values Before Filling:")
print(missing_values)

In [ ]:
# Specify the file path where you want to save the CSV file
output_file_path = "reduced_data.csv"

# Save the DataFrame to a CSV file
merged_data_new.to_csv(output_file_path, index=False)


In [ ]:
from sqlalchemy import create_engine

# Define the database connection URL. For SQLite, use a file path.
database_url = "sqlite:///sea_level_data.db" 

# Create a SQLAlchemy engine to connect to the database
engine = create_engine(database_url)

# Write the DataFrame to the database
reduced_data.to_sql("sea_level_data", con=engine, if_exists="replace", index=False)


In [ ]:
# Convert the 'Date' column to datetime format
merged_data_new['Date'] = pd.to_datetime(merged_data_new['Date'])

# Extract the year from the 'Date' column and add it as a new column
merged_data_new['Year'] = merged_data_new['Date'].dt.year

# Display the first few rows of the DataFrame to confirm the columns have been dropped
merged_data_new

In [ ]:
# Calculate the annual averages
annual_averages = merged_data_new.groupby('Year')['MSL (ft)'].mean()

# Display the annual averages
print("Annual Averages:")
print(annual_averages)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

X = merged_data_new[['Year']]
y = merged_data_new['MSL (ft)']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions for different time periods
years_to_predict = [10, 25, 50, 75, 100]

# Create a new DataFrame to hold the future years for prediction
future_years_df = pd.DataFrame({'Year': [year + merged_data_new['Year'].max() for year in years_to_predict]})

# Predict for future years
future_predictions = model.predict(future_years_df)

# Display the predictions
for year, prediction in zip(years_to_predict, future_predictions):
    print(f"Predicted Mean Coastal Sea Level in {year} years: {prediction:.2f} ft")

# Evaluate the model (for example, using Mean Absolute Error) on the test data
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error on Test Data: {mae:.2f} ft")

In [ ]:
# List of unique cities in your dataset
unique_cities = merged_data_new['City'].unique()

# Loop through each city
for city in unique_cities:
    # Filter the data for the current city
    city_data = merged_data_new[merged_data_new['City'] == city]

    # Extract the year and MSL (ft) for the current city
    X_city = city_data[['Year']]
    y_city = city_data['MSL (ft)']

    # Split the data into training and testing sets
    X_train_city, X_test_city, y_train_city, y_test_city = train_test_split(X_city, y_city, test_size=0.2, random_state=42)

    # Initialize and train the Linear Regression model for the current city
    model_city = LinearRegression()
    model_city.fit(X_train_city, y_train_city)

    # Make predictions for original and future years
    years_to_predict_city = list(city_data['Year']) + [year + city_data['Year'].max() for year in years_to_predict]
    predictions_city = model_city.predict([[year] for year in years_to_predict_city])

    # Create a plot to visualize the original data and predictions
    plt.figure(figsize=(10, 6))
    plt.scatter(X_city, y_city, label="Original Data", marker='o', color='blue')
    plt.plot(years_to_predict_city, predictions_city, label="Predicted Data", linestyle='--', color='red')
    plt.xlabel("Year")
    plt.ylabel("MSL (ft)")
    plt.title(f"{city} Sea Level Prediction")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Filter the data for San Francisco
san_francisco_data = merged_data_new[merged_data_new['City'] == 'San Francisco']

# Extract the year and mean highest columns
sf_years = san_francisco_data['Year']
sf_msl = san_francisco_data['MSL (ft)']

# Create a time series plot
plt.figure(figsize=(12, 6))
plt.plot(sf_years, sf_msl, marker='o', linestyle='-')
plt.title('Mean Highest Coastal Sea Level in San Francisco Over Time')
plt.xlabel('Year')
plt.ylabel('Mean Highest Sea Level (ft)')
plt.grid(True)
plt.show()

In [ ]:
# Filter the data for Boston
boston_data = merged_data_new[merged_data_new['City'] == 'Boston']

# Extract the year and mean highest columns
boston_years = boston_data['Year']
boston_msl = boston_data['Mean Highest']

# Create a time series plot
plt.figure(figsize=(12, 6))
plt.plot(years, mean_highest, marker='o', linestyle='-')
plt.title('Mean Highest Coastal Sea Level in Boston Over Time')
plt.xlabel('Year')
plt.ylabel('Mean Highest Sea Level (ft)')
plt.grid(True)
plt.show()

In [ ]:
# Filter the data for Charleston
charleston_data = reduced_data[reduced_data['City'] == 'Charleston']

# Extract the year and mean highest columns
years = charleston_data['Year']
mean_highest = charleston_data['Mean Highest']

# Create a time series plot
plt.figure(figsize=(12, 6))
plt.plot(years, mean_highest, marker='o', linestyle='-')
plt.title('Mean Highest Coastal Sea Level in Charleston Over Time')
plt.xlabel('Year')
plt.ylabel('Mean Highest Sea Level (ft)')
plt.grid(True)
plt.show()

In [ ]:
# Filter the data for Honolulu
honolulu_data = reduced_data[reduced_data['City'] == 'Honolulu']

# Extract the year and mean highest columns
years = honolulu_data['Year']
mean_highest = honolulu_data['Mean Highest']

# Create a time series plot
plt.figure(figsize=(12, 6))
plt.plot(years, mean_highest, marker='o', linestyle='-')
plt.title('Mean Highest Coastal Sea Level in Honolulu Over Time')
plt.xlabel('Year')
plt.ylabel('Mean Highest Sea Level (ft)')
plt.grid(True)
plt.show()

In [ ]:
# Filter the data for Honolulu
key_west_data = reduced_data[reduced_data['City'] == 'Key West']

# Extract the year and mean highest columns
years = key_west_data['Year']
mean_highest = key_west_data['Mean Highest']

# Create a time series plot
plt.figure(figsize=(12, 6))
plt.plot(years, mean_highest, marker='o', linestyle='-')
plt.title('Mean Highest Coastal Sea Level in Key West Over Time')
plt.xlabel('Year')
plt.ylabel('Mean Highest Sea Level (ft)')
plt.grid(True)
plt.show()

In [ ]:
# Filter the data for San Diego
san_diego_data = reduced_data[reduced_data['City'] == 'San Diego']

# Extract the year and mean highest columns
years = san_diego_data['Year']
mean_highest = san_diego_data['Mean Highest']

# Create a time series plot
plt.figure(figsize=(12, 6))
plt.plot(years, mean_highest, marker='o', linestyle='-')
plt.title('Mean Highest Coastal Sea Level in San Diego Over Time')
plt.xlabel('Year')
plt.ylabel('Mean Highest Sea Level (ft)')
plt.grid(True)
plt.show()

In [ ]:
# Filter the data for San Diego
san_diego_data = reduced_data[reduced_data['City'] == 'San Diego']

# Extract the year and mean highest columns
years = san_diego_data['Year']
mean_highest = san_diego_data['Mean Highest']

# Create a time series plot
plt.figure(figsize=(12, 6))
plt.plot(years, mean_highest, marker='o', linestyle='-')
plt.title('Mean Highest Coastal Sea Level in San Diego Over Time')
plt.xlabel('Year')
plt.ylabel('Mean Highest Sea Level (ft)')
plt.grid(True)
plt.show()